In [11]:
import IPython.core.display as di
# This line will add a button to toggle visibility of code blocks, for use with the HTML export version
di.display_html('''<button onclick="jQuery('.input_area').toggle(); jQuery('.prompt').toggle();">Toggle code</button>''', raw=True)


Toggle code

In [3]:
import numpy as np

class NaiveBayes:
    # Initialize an instance of the class.
    def __init__(self, alpha=0.0):
        self.alpha = alpha     # additive (Laplace) smoothing parameter
        self.priors = None     # estimated by fit()
        self.probs = None      # estimated by fit()
        self.num_labels = 0    # set by fit()
        self.num_features = 0  # set by fit()
        
    def fit(self, train_data, train_labels):
        # Store number of labels, number of features, and number training examples.
        self.num_labels = len(np.unique(train_labels))
        self.num_features = train_data.shape[1]
        self.num_examples = train_data.shape[0]
        
        # Initialize an array of label counts. Each label gets a smoothed count of 2*alpha because
        # each feature value (0 and 1) gets an extra count of alpha.
        label_counts = np.ones(self.num_labels) * self.alpha * 2

        # Initialize an array of (feature=1, label) counts to alpha.
        feature0_and_label_counts = np.ones([self.num_features, self.num_labels]) * self.alpha
    
        # Count features with value == 1.
        for i in range(self.num_examples):
            label = train_labels[i]
            label_counts[label] += 1
            for feature_index, feature_value in enumerate(train_data[i]):
                feature0_and_label_counts[feature_index][label] += (feature_value == 1)

        # Normalize to get probabilities P(feature=1|label).
        self.probs = feature0_and_label_counts / label_counts
        
        # Normalize label counts to get prior probabilities P(label).
        self.priors = label_counts / label_counts.sum()

    # Make predictions for each test example and return results.
    def predict(self, test_data):
        results = []
        for item in test_data:
            results.append(self._predict_item(item))
        return np.array(results)
    
    # Private function for making a single prediction.
    def _predict_item(self, item):
        # Make a copy of the prior probabilities.
        predictions = self.priors.copy()
        
        # Multiply by each conditional feature probability.
        for (index, value) in enumerate(item):
            feature_probs = self.probs[index]
            if not value: feature_probs = 1 - feature_probs
            predictions *= feature_probs

        # Normalize and return the label that gives the largest probability.
        predictions /= predictions.sum()
        return predictions.argmax()

In [4]:
from sklearn.datasets import load_iris
from sklearn.naive_bayes import BernoulliNB

iris = load_iris()
X,Y = iris.data,iris.target

#rearranging
np.random.seed(0)
shuffle = np.random.permutation(np.arange(X.shape[0]))
X,Y = X[shuffle], Y[shuffle]
#spliting into training and test data
train_data, train_labels = X[:100], Y[:100]
test_data, test_labels = X[100:], Y[100:]

def binarize_iris(data, threshold = [6.0,3.0,2.5,1.0]):
    binarized = np.zeros(data.shape)
    for feature in range(data.shape[1]):
        binarized[:,feature] = data[:,feature] > threshold[feature]
    return binarized

binarized_train_data = binarize_iris(train_data)
binarized_test_data = binarize_iris(test_data)

In [6]:
def perform_comparison(alpha):
    nby = NaiveBayes(alpha=alpha)
    nby.fit(binarized_train_data, train_labels)

    # Compute accuracy on the test data.
    preds = nby.predict(binarized_test_data)
    correct, total = 0, 0
    for pred, label in zip(preds, test_labels):
        if pred == label: correct += 1
        total += 1
    print('With alpha = %.2f' %alpha)
    print('[OUR implementation] total: %3d  correct: %3d  accuracy: %3.2f' %(total, correct, 1.0*correct/total))




    # Compare to sklearn's implementation.
    clf = BernoulliNB(alpha=alpha, fit_prior=False)
    clf.fit(binarized_train_data, train_labels)
    print('sklearn accuracy: %3.2f' %clf.score(binarized_test_data, test_labels))

    print('\nOur feature probabilities\n', nby.probs)
    print('\nsklearn feature probabilities\n', np.exp(clf.feature_log_prob_).T)
    print('\nOur prior probabilities\n', nby.priors)
    print('\nsklearn prior probabilities\n', np.exp(clf.class_log_prior_))

In [10]:
from ipywidgets import widgets
from IPython.display import display
from IPython.display import clear_output
num = widgets.BoundedIntText(description = "Laplace Alpha")

button = widgets.Button(description="Compare!")
display(num)
display(button)

 
def on_button_clicked(sender):
    clear_output()
    perform_comparison(num.value)

button.on_click(on_button_clicked)

With alpha = 20.00
[OUR implementation] total:  50  correct:  41  accuracy: 0.82
sklearn accuracy: 0.82

Our feature probabilities
 [[ 0.28169014  0.45205479  0.64473684]
 [ 0.63380282  0.35616438  0.40789474]
 [ 0.28169014  0.7260274   0.73684211]
 [ 0.28169014  0.68493151  0.73684211]]

sklearn feature probabilities
 [[ 0.28169014  0.45205479  0.64473684]
 [ 0.63380282  0.35616438  0.40789474]
 [ 0.28169014  0.7260274   0.73684211]
 [ 0.28169014  0.68493151  0.73684211]]

Our prior probabilities
 [ 0.32272727  0.33181818  0.34545455]

sklearn prior probabilities
 [ 0.33333333  0.33333333  0.33333333]
